In [ ]:
import os
from glob import glob
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix
import keras
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.layers import Activation,Dense, Dropout, Flatten

import datetime
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from keras.models import Model

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
imageSize=128
train_dir = "/content/drive/MyDrive/COI/Train"
# ['Covid19', 'Normal', 'Opacity', 'Pneumonia']
from tqdm import tqdm
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):

            if folderName in ['Covid']:
                label = 0
            elif folderName in ['Normal']:
                label = 1

                '''
            elif folderName in ['Covid']:
                label = 2
            elif folderName in ['Normal]:
                label = 3
                '''
            else:
                label = 4

            for image_filename in tqdm(os.listdir(folder + '/'+folderName)):


                img_file = cv2.imread(folder + '/'+folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)

                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)

    return X,y
X_train, y_train = get_data(train_dir) # Un-comment to use full dataset: Step 1 of 2



100%|██████████| 1864/1864 [01:09<00:00, 27.00it/s]


In [ ]:
X_train.shape

(3728, 128, 128, 3)

In [ ]:
from tensorflow.keras.applications import EfficientNetV2L

In [ ]:
''' Inception Model '''
model1 = tf.keras.applications.EfficientNetV2L(input_shape=(128,128,3),include_top=False,weights='imagenet',pooling='avg')

''' freezing layers '''
model1.trainable = False

473176280/473176280 [==============================] - 23s 0us/step


In [ ]:
model1.input
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense

In [ ]:
label_input = Input(shape=(), dtype=tf.int64)  # Assuming label is a scalar


In [ ]:
x = Flatten()(model1.output)

# and a sofymax/logistic layer -- we have 6 classes
predictions = Dense(3, activation='softmax')(x)
# create a model object
model = Model(inputs=[model1.input,label_input], outputs=predictions)

In [ ]:

model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 3)  0           ['input_1[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 64, 64, 32)   864         ['rescaling[0][0]']              
                                                                                                  
 stem_bn (BatchNormalization)   (None, 64, 64, 32)   128         ['stem_conv[0][0]']        

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
np.unique(y_train)

array([0, 1, 4])

In [ ]:
history = model.fit(
  (X_train,y_train),
  validation_data=(X_test,y_test),
  epochs=100
)

Epoch 1/100


ValueError: ignored